In [2]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models.forecasting.tft_model import TFTModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, mae, mse
from torchmetrics.regression import MeanAbsoluteError
from darts.utils import missing_values
from sklearn.preprocessing import MinMaxScaler
from darts.utils.likelihood_models import QuantileRegression
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from darts.explainability.tft_explainer import TFTExplainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.DataFrame(pd.read_csv('../raw_data/preprocessed_from_may_2022.csv')).set_index('Unnamed: 0')

# Further processing for TFT model.
df.index = pd.to_datetime(df.index)
df.interpolate(method='linear', inplace=True)

target = df['rate_scaled']
past_cov = df.drop(columns=['rate', 'rate_scaled'])
future_cov = pd.DataFrame(df.index.to_series().dt.dayofweek)

future_scaler = MinMaxScaler()
future_cov = pd.DataFrame(future_scaler.fit_transform(future_cov))
future_cov.index = df.index

# Defining train, test, val.
test = 24
train = int(0.8*len(df))
val = len(df) - train -test

# Splitting data between future and past for train, test and val.
y_train = target[-train-val-test:- test]
future_cov_train = future_cov[-train-val-test:]
past_cov_train = past_cov[-train-val-test:-test]
y_val = target[-test:]

# Transforming the Panda series and dataframes into Darts TimeSeries
y_train_series = TimeSeries.from_series(y_train)
past_cov_train_series = TimeSeries.from_dataframe(past_cov_train)
future_cov_train_series = TimeSeries.from_dataframe(future_cov_train)

y_val_series = TimeSeries.from_series(y_val)
# past_cov_val_series = TimeSeries.from_dataframe(past_cov_val)
# future_cov_val_series = TimeSeries.from_dataframe(future_cov_val)

# y_test_series = TimeSeries.from_series(y_test)

In [7]:
# Input is 5 days (in hours), output is 24 hours.
input_chunk_length = (5*24)
output_chunk_length = 24

my_stopper = EarlyStopping(
            monitor="train_loss",
            patience=5,
            min_delta=0.001,
            mode='min',
        )

pl_trainer_kwargs={"callbacks": [my_stopper],
                   "accelerator": "cpu"}

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau

# model = TFTModel(input_chunk_length =input_chunk_length,
#        output_chunk_length = output_chunk_length,
#         pl_trainer_kwargs=pl_trainer_kwargs,
#        lstm_layers=2,
#        num_attention_heads=4,
#        dropout=0.3,
#        batch_size=16,
#        hidden_size=64,
#        loss_fn=torch.nn.MSELoss(),
#        n_epochs=1,
#        likelihood=None,
#        lr_scheduler_cls = lr_scheduler,
#        random_state=42,
#       )

In [8]:
model = TFTModel(input_chunk_length =input_chunk_length ,
               output_chunk_length = output_chunk_length,
               pl_trainer_kwargs = pl_trainer_kwargs,
               lstm_layers=2,
               num_attention_heads=8,
               dropout=0.2,
               batch_size=16,
               hidden_size=32,
               torch_metrics=MeanAbsoluteError(),
               lr_scheduler_cls = lr_scheduler,
               n_epochs=50
               )

In [9]:
hyperparameters = {
'input_chunk_length': [input_chunk_length],
'output_chunk_length': [output_chunk_length],
'lstm_layers' : [1, 2],
'n_epochs' : [50],
'random_state': [0, 42],
'hidden_size': [10, 20, 32],
'num_attention_heads': [4, 8],
'dropout' : [0.2, 0.3, 0.4],
'batch_size': [16, 32]
}

In [12]:
best_model

(TFTModel(output_chunk_shift=0, hidden_size=32, lstm_layers=1, num_attention_heads=8, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.2, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=False, loss_fn=None, likelihood=None, norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=120, output_chunk_length=24, n_epochs=50, random_state=0, batch_size=16),
 {'input_chunk_length': 120,
  'output_chunk_length': 24,
  'lstm_layers': 1,
  'n_epochs': 50,
  'random_state': 0,
  'hidden_size': 32,
  'num_attention_heads': 8,
  'dropout': 0.2,
  'batch_size': 16},
 3.4839319113023506)